# TODO: work on pred_dann so it adapts to the new architecture

In [1]:
import numpy as np
np.random.seed(0) 

import argparse

import os
import math
from math import sin

import json

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import seaborn as sns

import time

import wandb

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
matplotlib.rc( 'savefig', facecolor = 'white' )
from matplotlib import pyplot
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchsummary import summary
torch.manual_seed(0)

i_seed = 0

from datetime import datetime
import pytz
import pprint

import sys
sys.path.append('../') # add this line so Data and data are visible in this file
sys.path.append('../../') # add this line so Data and data are visible in this file
sys.path.append('../PatchWand/') # add this line so Data and data are visible in this file

# from PatchWand import *
from plotting_tools import *
from handy_tools import *
from setting import *
from evaluate import *

from stage3_preprocess import *
from stage4_regression import *
from dataIO import *
from FL_extension.training_util import *
from FL_extension.dataset_util import *
from FL_extension.evaluation_util import *
from FL_extension.models import *
from FL_extension.models_CNNlight import *
# from unet_extension.training_util import *

from importlib import reload
%load_ext autoreload
%autoreload 2

In [2]:
print(torch.version.cuda)

10.2


In [3]:
parser = argparse.ArgumentParser(description='SpO2_estimate')
parser.add_argument('--input_folder', metavar='input_folder', help='input_folder',
                    default='../')
parser.add_argument('--output_folder', metavar='output_folder', help='output_folder',
                    default='../')
parser.add_argument('--subject_id', metavar='subject_id', help='subject_id',
                    default='101')
parser.add_argument('--training_params_file', metavar='training_params_file', help='training_params_file',
                    default='training_params_list.json')
parser.add_argument('--use_denoise_scg', metavar='use_denoise_scg', help='use_denoise_scg',
                    default='true')


# checklist 3: comment first line, uncomment second line
# args = parser.parse_args(['--input_folder', '../../data/stage3_FL/win8_overlap75', 
#                           '--output_folder', '../../data/stage4_FL/',
# #                           '--training_params_file', 'training_params_baseline.json',
#                           '--training_params_file', 'training_params_dummy.json',
#                          ])
args = parser.parse_args(['--input_folder', '../../data/stage3/win60_overlap90/', 
                          '--output_folder', '../../data/stage4_FL/',
#                           '--training_params_file', 'training_params_baseline.json',
#                           '--training_params_file', 'training_params_dummy.json',
#                           '--training_params_file', 'training_params_STFT.json',
                          '--training_params_file', 'training_params_computation.json',
#                           '--training_params_file', 'training_params_RespiratoryRegression.json',
                          '--use_denoise_scg', 'True',
                         ])
# args = parser.parse_args()
print(args)

Namespace(input_folder='../../data/stage3/win60_overlap90/', output_folder='../../data/stage4_FL/', subject_id='101', training_params_file='training_params_computation.json', use_denoise_scg='True')


In [4]:
# if 1==1 :
#     print('hi')

# start timing

In [5]:

tz_NY = pytz.timezone('America/New_York') 
datetime_start = datetime.now(tz_NY)
print("start time:", datetime_start.strftime("%Y-%b-%d %H:%M:%S"))



start time: 2023-May-17 02:40:04


In [6]:
inputdir = args.input_folder
outputdir = args.output_folder
training_params_file = args.training_params_file

use_denoise_scg = eval(args.use_denoise_scg)

outputdir = outputdir + '{}/'.format(training_params_file.split('.json')[0].split('_')[-1])


if not os.path.exists(outputdir):
    os.makedirs(outputdir)

# get training params and dataloaders

In [7]:
torch.cuda.is_available()

True

In [8]:
def get_model_out_names(training_params):
    model_out_names = []

#     for output_name in training_params['output_names']:
    for output_name in training_params['output_names']+['domain']:
        for input_name in training_params['input_names']:
            model_out_names.append(output_name+'-{}'.format(input_name))
    return model_out_names

In [9]:
def get_modality_dict(training_params):
    label_encoder = LabelEncoder()
    modality_encoded = label_encoder.fit_transform(training_params['input_names'])
#     modality_encoded # array([0, 1])

    modality_dict = {}
    for i_modality in modality_encoded:
        modality_dict[training_params['input_names'][i_modality]] = i_modality
#     training_params['modality_dict'] = modality_dict
    return modality_dict

In [10]:
training_params_file

'training_params_computation.json'

In [11]:
with open(training_params_file) as json_file:
    training_params_list = json.load(json_file)

for training_params in [training_params_list[0]]:
    
    # 1. store device info
    # include device in training_params
    if training_params['cuda_i']==-1:
        device = torch.device('cpu')
    else:
        device = torch.device('cuda:{}'.format(int(training_params['cuda_i'])) if torch.cuda.is_available() else 'cpu')
    training_params['device'] = device
    
    # 2. check training mode (subject_ind [default] vs. subject_specific)
    if 'training_mode' in training_params:
        training_mode = training_params['training_mode']
    else:
        training_params = 'subject_ind'
    
    # 3. transfer some info from stage3_dict to training_params
    stage3_dict = data_loader('stage3_dict', inputdir).item()
    training_params['list_signal'] = stage3_dict['list_signal']
    training_params['list_feature'] = stage3_dict['list_feature']
    training_params['list_output'] = stage3_dict['list_output']
    training_params['list_meta'] = stage3_dict['list_meta']
    training_params['FS_RESAMPLE_DL'] = stage3_dict['FS_RESAMPLE_DL']
    training_params['subject_ids'] = stage3_dict['subject_ids']
    training_params['task_ids'] = stage3_dict['task_ids']
    training_params['sequence'] = stage3_dict['sequence']
    
    # 4. get data loaders (TODO: load data into script once to save IO time)
    # [change it] first change CV_config so get_loaders prepare the data correctly though
    training_params['CV_config'] = {
        'subject_id': 113,
#         'task_id': 5,
    }
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    print('data dimensions are:', dataloaders['train'].dataset.data.shape)
    print('# of windows:', dataset_sizes['train']+dataset_sizes['val'])
    
    data_dimensions = dataloaders['train'].dataset.__getitem__(0)[0].size()
    training_params['data_dimensions'] = list(data_dimensions)
    del dataloaders
    
    # 5. get sweep name (is this necessary?)
    sweep_name = training_params['sweep_name'] 
    


    # 7. store the ordered_subject_ids, inputdir, and outputdir
    training_params['ordered_subject_ids'] = np.asarray(training_params['ordered_subject_ids'])
#       "ordered_subject_ids": [101, 103, 104, 106, 107, 110, 111, 115, 116, 117, 118, 119, 120, 121, 113],
    # [101, 103, 104, 106, 111, 115, 116, 117, 118, 119, 120, 121, 113],
    
    training_params['inputdir'] = inputdir
    training_params['outputdir'] = outputdir


# training_params = training_params_list[0]

data dimensions are: (417, 1, 6000)
# of windows: 438


In [12]:
# dataloaders, dataset_sizes = get_loaders(inputdir, training_params)


In [13]:
# dataloaders

In [14]:
def denoise_scg(sig, Fs):

    sig_analytic = hilbert(sig)
    sig_envelope = np.abs(sig_analytic)
    sig_filt = get_padded_filt(sig_envelope, lowcutoff=0.1, highcutoff=10, Fs=Fs)

    return sig_filt


In [15]:
df_outputlabel_val = pd.DataFrame()
df_performance_val = pd.DataFrame()

for i_CV, subject_id in enumerate(training_params['ordered_subject_ids']):
    print('working on subject', subject_id)
    if 'CV_max' in training_params:
        if i_CV >= training_params['CV_max']:
            continue

    training_params['CV_config']['subject_id'] = subject_id

    # need to load the data for each LOSO CV
    dataloaders, dataset_sizes = get_loaders(inputdir, training_params)
    print('data dimensions are:', dataloaders['val'].dataset.data.shape)
    print('dataset_sizes: ', dataset_sizes)

    # update the dimension so the model is created correctly
    data_dimensions = dataloaders['val'].dataset.__getitem__(0)[0].size()
    training_params['data_dimensions'] = list(data_dimensions)


    for j in range(dataloaders['val'].dataset.__len__()):
                   
        HR_label = dataloaders['val'].dataset.label[j,0]
        sig = dataloaders['val'].dataset.data[j,0,:]
        
        if use_denoise_scg:
            sig = denoise_scg(sig, Fs=training_params['FS_RESAMPLE_DL'])


        xf, yf = get_psd(sig, Fs=training_params['FS_RESAMPLE_DL'])

        xf = xf * 60
        mask = (xf>=label_range_dict['HR_DL'][0]) & (xf<=label_range_dict['HR_DL'][1])
        xf = xf[mask]
        yf = yf[mask]

        i_max = np.argmax(yf)
        HR_est = xf[i_max]
        HR_est, HR_label
                   
        df = pd.DataFrame( 
            {
                'label_est': HR_est,
                'label': HR_label,
                'CV': subject_id,
                'task': 'HR_estimation',
                'activity': np.vectorize(tasks_dict_reversed.get)(dataloaders['train'].dataset.meta[i,1])
            }, index=[0] )
                   
        df_outputlabel_val = df_outputlabel_val.append(df)

    df_outputlabel_sub = df_outputlabel_val[df_outputlabel_val['CV']==subject_id]
    df_performance_val = df_performance_val.append( get_df_performance(df_outputlabel_sub['label'].values, df_outputlabel_sub['label_est'].values, subject_id, 'HR_estimation'), ignore_index=True )


working on subject 107
data dimensions are: (21, 1, 6000)
dataset_sizes:  {'train': 417, 'val': 21}
working on subject 121
data dimensions are: (20, 1, 6000)
dataset_sizes:  {'train': 418, 'val': 20}
working on subject 117
data dimensions are: (21, 1, 6000)
dataset_sizes:  {'train': 417, 'val': 21}
working on subject 101
data dimensions are: (20, 1, 6000)
dataset_sizes:  {'train': 418, 'val': 20}
working on subject 102
data dimensions are: (22, 1, 6000)
dataset_sizes:  {'train': 416, 'val': 22}
working on subject 103
data dimensions are: (21, 1, 6000)
dataset_sizes:  {'train': 417, 'val': 21}
working on subject 104
data dimensions are: (20, 1, 6000)
dataset_sizes:  {'train': 418, 'val': 20}
working on subject 105
data dimensions are: (20, 1, 6000)
dataset_sizes:  {'train': 418, 'val': 20}
working on subject 106
data dimensions are: (21, 1, 6000)
dataset_sizes:  {'train': 417, 'val': 21}
working on subject 108
data dimensions are: (21, 1, 6000)
dataset_sizes:  {'train': 417, 'val': 21}


In [16]:
df_performance_val['mae'].mean(), df_performance_val['mae'].std()

(4.260116337254515, 6.162864422844996)

In [17]:
df_performance_val['rmse'].mean(), df_performance_val['rmse'].std()

(5.834240727557825, 9.077831237986317)

In [ ]:

fig, axes = plt.subplots(2,1,figsize=(5,7), dpi=150)


j = 5

sig = dataloaders['val'].dataset.data[j, 0, :]
HR_label = dataloaders['val'].dataset.label[j,0]
xf, yf = get_psd(sig, Fs=training_params['FS_RESAMPLE_DL'])
axes[0].plot(xf, yf)


def denoise_scg(sig, Fs):

    sig_analytic = hilbert(sig)
    sig_envelope = np.abs(sig_analytic)
    sig_filt = get_padded_filt(sig_envelope, lowcutoff=0.1, highcutoff=None, Fs=Fs)

    return sig_filt



# sig_analytic = hilbert(sig)
# sig_envelope = np.abs(sig_analytic)
# plt.plot(sig)

sig_filt = denoise_scg(sig, Fs=training_params['FS_RESAMPLE_DL'])
xf, yf = get_psd(sig_filt, Fs=training_params['FS_RESAMPLE_DL'])
yf = get_smooth(yf, N=7)

axes[1].plot(xf, yf, color='steelblue', linewidth=2)

mask = (xf>=1.3) & (xf<=1.7)
axes[1].fill_between(xf[mask], yf[mask], alpha=0.8, color='steelblue')

for ax in axes:
    ax.set_xlim(0,5)
    ax.set_ylim(0, ax.get_ylim()[1])
    ax.set_xlabel('freq (Hz)')

In [ ]:
# plt.plot(sig_envelope)

In [ ]:

xf = xf * 60
# mask = (xf>=label_range_dict['HR_DL'][0]) & (xf<=label_range_dict['HR_DL'][1])
# xf = xf[mask]
# yf = yf[mask]

i_max = np.argmax(yf)
HR_est = xf[i_max]
HR_est, HR_label


# axes[1].plot(amplitude_envelope)
# axes[2].plot(signal_filt)

In [ ]:
# fig, axes = plt.subplots(3,1,figsize=(30,3), dpi=100)
# axes[0].plot(signal)
# axes[1].plot(amplitude_envelope)
# axes[2].plot(signal_filt)

In [ ]:
dataloaders

In [ ]:
 get_df_performance(df['label'].values, df['label_est'].values, subject_id, 'HR_estimation'), ignore_index=True )


In [ ]:
aaa = df_outputlabel_val['label_est']-df_outputlabel_val['label']
np.mean(np.abs(aaa))

In [ ]:
plt.plot(df_outputlabel_val['label_est'].values)
plt.plot(df_outputlabel_val['label'].values)

In [ ]:
# get_df_performance(df_outputlabel_val['label'], df_outputlabel_val['label_est'])

In [ ]:
# df_outputlabel_val['label_est'].values.shape

In [ ]:
plt.plot(sig)

In [ ]:
training_params['model_out_names']

In [ ]:
training_params['regressor_names']